# Downloading the dataset

### Import libraries

In [2]:
import requests, zipfile, io
from typing import List, Dict
from requests.adapters import HTTPAdapter, Retry
import random
import json
import os
import pandas as pd
from codecarbon import EmissionsTracker

### Endpoints

In [3]:
URL = "http://s3-ceatic.ujaen.es:8036/" 
TOKEN = "faae943b33e3f5947d122fd9ebca17727fbfbce5" 

# Download endpoints
ENDPOINT_DOWNLOAD_MESSAGES_TRIAL = URL+"{TASK}/download_trial/{TOKEN}"
ENDPOINT_DOWNLOAD_GOLD_TRIAL = URL+"{SUBTASK}/download_trial/{TOKEN}"
ENDPOINT_DOWNLOAD_MESSAGES_TRAIN = URL+"{TASK}/download_train/{TOKEN}"
ENDPOINT_DOWNLOAD_GOLD_TRAIN = URL+"{SUBTASK}/download_train/{TOKEN}"

# Trial endpoints
ENDPOINT_GET_MESSAGES_TRIAL = URL+"{TASK}/getmessages_trial/{TOKEN}"
ENDPOINT_SUBMIT_DECISIONS_TRIAL = URL+"{SUBTASK}/submit_trial/{TOKEN}/{RUN}"

# Test endpoints
ENDPOINT_GET_MESSAGES = URL+"{TASK}/getmessages/{TOKEN}"
ENDPOINT_SUBMIT_DECISIONS = URL+"{SUBTASK}/submit/{TOKEN}/{RUN}"

### Download data 

In [4]:
def download_messages_trial(task: str,subtasks:List[str], token: str) -> List[Dict]:
    response = requests.get(ENDPOINT_DOWNLOAD_MESSAGES_TRIAL.format(TASK=task, TOKEN=token))

    if response.status_code != 200:
        print("Trial - Status Code " + task + ": " + str(response.status_code) + " - Error: " + str(response.text))
    else:
      z = zipfile.ZipFile(io.BytesIO(response.content))
      os.makedirs("./data/{task}/trial/subjects_trial/".format(task=task))
      z.extractall("./data/{task}/trial/subjects_trial/".format(task=task))

    for subtask in subtasks:
        response = requests.get(ENDPOINT_DOWNLOAD_GOLD_TRIAL.format(SUBTASK=subtask, TOKEN=token))
        
        if response.status_code != 200:
            print("Trial - Status Code " + subtask + ": " + str(response.status_code) + " - Error: " + str(response.text))
        else:
          file_object = open("./data/{task}/trial/gold_trial_{subtask}.txt".format(task=task, subtask=subtask), "w")
          file_object.write(response.text)

def download_messages_train(task: str,subtasks:List[str], token: str) -> List[Dict]:
    response = requests.get(ENDPOINT_DOWNLOAD_MESSAGES_TRAIN.format(TASK=task, TOKEN=token))

    if response.status_code != 200:
        print("Trial - Status Code " + task + ": " + str(response.status_code) + " - Error: " + str(response.text))
    else:
      z = zipfile.ZipFile(io.BytesIO(response.content))
      os.makedirs("./data/{task}/trial/subjects_train/".format(task=task))
      z.extractall("./data/{task}/trial/subjects_train/".format(task=task))

    for subtask in subtasks:
        response = requests.get(ENDPOINT_DOWNLOAD_GOLD_TRAIN.format(SUBTASK=subtask, TOKEN=token))
        
        if response.status_code != 200:
            print("Trial - Status Code " + subtask + ": " + str(response.status_code) + " - Error: " + str(response.text))
        else:
          file_object = open("./data/{task}/trial/gold_train_{subtask}.txt".format(task=task, subtask=subtask), "w")
          file_object.write(response.text)

def download_messages_test(task: str,subtasks:List[str], token: str) -> List[Dict]:
    response = requests.get(ENDPOINT_DOWNLOAD_MESSAGES_TRAIN.format(TASK=task, TOKEN=token))

    if response.status_code != 200:
        print("Trial - Status Code " + task + ": " + str(response.status_code) + " - Error: " + str(response.text))
    else:
      z = zipfile.ZipFile(io.BytesIO(response.content))
      os.makedirs("./data/{task}/trial/subjects_test/".format(task=task))
      z.extractall("./data/{task}/trial/subjects_test/".format(task=task))

### Server client

In [5]:
class Client_taskX:
    def __init__(self, task: str, subtasks: List[str], token: str, number_of_runs: int, tracker: EmissionsTracker):
        # Task in which you participate
        self.task = task
        # Subtasks in which you participate
        self.subtasks = subtasks
        # Token identifier
        self.token = token
        # Number of runs (Max: 3)
        self.number_of_runs = number_of_runs
        # Object to calculate CO2 emissions
        self.tracker = tracker
        self.relevant_cols = ['duration', 'emissions', 'cpu_energy', 'gpu_energy', 'ram_energy', 
            'energy_consumed', 'cpu_count', 'gpu_count', 'cpu_model', 'gpu_model', 'ram_total_size']

    # Here a GET request is sent to the server to extract the data.
    def get_messages(self, retries: int, backoff: float) -> Dict:
        session = requests.Session()
        retries = Retry( 
                        total = retries,
                        backoff_factor = backoff,
                        status_forcelist = [500, 502, 503, 504]
                        )
        session.mount('https://', HTTPAdapter(max_retries=retries))
        response = session.get(ENDPOINT_GET_MESSAGES_TRIAL.format(TASK=self.task, TOKEN=self.token))
        if response.status_code != 200:
          print("GET - Status Code " + self.task + ": " + str(response.status_code) + " - Error: " + str(response.text))
          return []
        else:
          return json.loads(response.content)

    # The POST requests are sent to the server to send predictions and carbon emission data
    def submit_decission(self, subtask: int, messages: List[Dict], emissions:Dict, retries, backoff):
        decisions = {}

        # You must create the appropriate structure to send the predictions according to each subtask
        for message in messages:
            decisions[message["nick"]] = random.randint(0, 1) # the decision of your system according to subtask

        data = {
            "predictions": decisions,
            "emissions": emissions
        }

        data = json.dumps(data)
        # Session to POST request
        session = requests.Session()
        retries = Retry(
                        total = retries,
                        backoff_factor = backoff,
                        status_forcelist = [500, 502, 503, 504]
                        )
        session.mount('https://', HTTPAdapter(max_retries=retries))

        for run in range(0,self.number_of_runs):
            # For each run, new decisions
            response = session.post(ENDPOINT_SUBMIT_DECISIONS_TRIAL.format(SUBTASK=self.subtasks[subtask], TOKEN=self.token, RUN=run), json=[data])
            if response.status_code != 200:
                print("POST - Status Code " + self.task + ": " + str(response.status_code) + " - Error: " + str(response.text))
            else:
                print("Subtask {}: - run {}".format(self.subtasks[subtask], run))
        

    # Main thread
    def run_taskX(self, retries: int, backoff: float):
        # Get messages for taskX
        messages = self.get_messages(retries, backoff)
        # If there are no messages
        if len(messages) == 0:
            print("All rounds processed")
            return

        while len(messages) > 0:
            print("------------------- Processing round {}".format(messages[0]["round"]))
            # Save subjects
            with open('./data/rounds_trial/round{}.json'.format(messages[0]["round"]), 'w+', encoding='utf8') as json_file:
                json.dump(messages, json_file, ensure_ascii=False)

            """# Calculate emissions for each prediction
            self.tracker.start()

            # Your code
            

            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()

            self.submit_decission(0, messages, measurements, retries, backoff) # taskXa
            self.submit_decission(1, messages, measurements, retries, backoff) # taskXb
            self.submit_decission(2, messages, measurements, retries, backoff) # taskXc
            self.submit_decission(3, messages, measurements, retries, backoff) # taskXd
            
            # Only one GET request for each round
            messages = self.get_messages(retries, backoff)
            """
        print("All rounds processed")

### Downloading

In [6]:
def download_data():
    download_messages_trial("task2", ["task2a"], TOKEN)
    download_messages_train("task2", ["task2a"], TOKEN)

def get_post_data():
    # Emissions Tracker Config
    config = {
        "save_to_file": True,
        "log_level": "DEBUG",
        "tracking_mode": "process",
        "output_dir": ".", 
    }
    tracker = EmissionsTracker(**config)

    number_runs = 3 # Max: 3

    # Prediction period
    client_taskX = Client_taskX("task2", ["task2a"], TOKEN, number_runs, tracker)
    client_taskX.run_taskX(5, 0.1)

In [7]:
#download_data()
#get_post_data()

FileExistsError: [Errno 17] File exists: './data/task2/trial/subjects_trial/'

In [8]:
download_messages_test("task2", ["task2a"], TOKEN)